# Coupling CEM and HydroTrend to assess links between rivers and coastal morphology
In this lab, you will use a coupled Coastal Evolution Model (CEM) and HydroTrend model to study the effects of coastal river drainage discharge on sandy shoreline change through time. 
* Hydrotrend is a 2D numerical model that operates using a water and sediment budget and delivers a daily sediment output to a river outlet. Read more about it here: https://csdms.colorado.edu/wiki/Model:HydroTrend.
* CEM, on the other hand, is a 2D numerical model that addresses wave-driven morphological change undergoing sandy coastal systems. Read more about it here: http://csdms.colorado.edu/wiki/Model_help:CEM   

### Parameters of interest
Hydrotrend requires inputs of various watershed-specific information including hypsometric, climatic, lithologic, and anthropogonic data.  It creates time-step specific temperature and precipitation scenarios that affect weathering and sediment transport rates across the drainage basin 
and gives an output of daily sediment loads. For this lab, we have included the past 20 years of climatic data. 

CEM requires wave-climate data and shoreline position inputs and solves for sediment transport and shoreline morphological change including spit formation and erosion. 

### Case Study: the effects of the Coquilles River on the southern Oregon coastline
The Coquilles River is a ~60-km long river that drains 2,740 $\text{km}^{2}$ of the Coast range in Oregon. Over the past century, it has experienced intense anthropogenic disturbances including channel dredging, gravel bar mining, and deforestoration. <u>This lab explores the impact of anthropogenic and climate-change disturbances within the watershed on coastline change.</u>  

### Links
* Link to this notebook: 
* Package installation commands: 
* Command to download a local copy: 

### Key References

Ashton, A.D.; Hutton, E.W.H.; Kettner, A.J.; Xing, F.; Kallumadikal, J.; Nienhuis, J.; Giosan, L. Progress in coupling models of coastline and fluvial dynamics. Comput. Geosci. 2013, 53, 21–29.

Warrick, J. A., Vos, K., East, A. E. & Vitousek, S. (2022). Fire (plus) flood (equals) beach: coastal response to an exceptional river sediment discharge event. Scientific Reports, 12(1), 3848. https://doi.org/10.1038/s41598-022-07209-0

## Visualizing Coquille River watershed elevation
First you will import the hypsometry (i.e., elevation distribution). You could visualize your own watershed dynamics using this lab: 

## Parameterizing Hydrotrend to Coquilles River watershed

## Importing CoastSTAT shoreline and wave climates

## Running CEM-Hydrotrend

## How does the coastline change with varying sediment load frequency and magnitude? 